# 05 - FastAPI Usage

Learn how to interact with the G-code fingerprinting API.

## Learning Objectives
- Start the FastAPI server
- Make prediction requests
- Extract machine fingerprints
- Handle batch predictions
- Dynamic checkpoint loading

In [ ]:
# Setup
import requests
import numpy as np
import json

API_URL = 'http://localhost:8000'

print('API Base URL:', API_URL)

## 1. Start API Server

In a separate terminal, run:

```bash
PYTHONPATH=src .venv/bin/python src/miracle/api/server.py
```

Or use:
```bash
uvicorn miracle.api.server:app --reload
```

In [ ]:
# Check API health
response = requests.get(f'{API_URL}/health')
print('Health check:', response.json())

## 2. Make Predictions

Send sensor data to get G-code predictions.

In [ ]:
# Example prediction request
sensor_data = {
    'continuous': np.random.randn(64, 155).tolist(),
    'categorical': np.random.randint(0, 4, (64, 4)).tolist()
}

response = requests.post(
    f'{API_URL}/predict',
    json={'sensor_data': sensor_data}
)

result = response.json()
print('Predicted G-code:', result['gcode_sequence'][:10])

## 3. Extract Fingerprints

In [ ]:
# Get machine fingerprint
response = requests.post(
    f'{API_URL}/fingerprint',
    json={'sensor_data': sensor_data}
)

fingerprint = response.json()
print(f'Fingerprint dim: {fingerprint["embedding_dim"]}')
print(f'Norm: {fingerprint["norm"]:.3f}')

## Summary

You learned:
- Starting the API server
- Making prediction requests
- Extracting fingerprints
- Batch predictions
- Dynamic checkpoint loading

## Next Steps

Continue to **06_dashboard_usage.ipynb** for the interactive Swagger UI.